In [23]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math

import torch
import torch.nn as nn
from torch.optim import RMSprop

from torch.utils.data import DataLoader, TensorDataset

In [167]:
data = pd.read_csv('../data/jhu-covid-cases.csv')
data = data[data.Province_State == 'Montana']
data = data[data.columns[11:]]
print(data.columns[:5])

death = pd.read_csv('../data/jhu-covid-deaths.csv')
death = death[death.Province_State == 'Montana']
death = death[death.columns[12:]]
print(death.columns[:5]) # make sure the dates match

data = np.array(data.sum(axis = 0)) #.reshape(-1, 1)
data = np.convolve(data, np.ones(7) / 7, 'valid')
data = torch.tensor(data)
Itilde = torch.log(1 + data[20:] - data[:-20]).reshape(1, -1)

death = torch.tensor(death.sum(axis = 0))
Dt = torch.tensor(np.convolve(death, np.ones(7) / 7, 'valid'))
Dtilde = torch.log(1 + Dt[:-20]).reshape(1, -1)

N = 1_123_000 # montana population 2022 (assume constant)
Ntilde = math.log(1 + N)

Stilde = torch.log(N - torch.exp(Itilde) - torch.exp(Dtilde))

first_index = torch.arange(1117)[Itilde[0,:] > 4][0].item() # first index above 50 cases (log(50) is 4 basically)

Stilde = Stilde[:,first_index:].T
Itilde = Itilde[:,first_index:].T
Dtilde = Dtilde[:,first_index:].T

Y = torch.hstack((Stilde, Itilde, Dtilde))
T = torch.arange(1078).reshape(-1, 1) / 1078 

print(T.shape, Y.shape) # make sure shapes match 

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20'], dtype='object')
Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20'], dtype='object')
torch.Size([1078, 1]) torch.Size([1078, 3])


In [184]:
class Greek(nn.Module): # 1 to 1 (time to positive value)
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(1, 16)
        self.lin2 = nn.Linear(16, 16)
        self.lin3 = nn.Linear(16, 16)
        self.lin4 = nn.Linear(16, 1)

        self.act = torch.relu

        #self.param = nn.Parameter(torch.randn(1078, 1))

    #"""
    def forward(self, x):
        x = self.lin1(x)
        x = self.act(x)
        x = self.lin2(x)
        x = self.act(x)
        x = self.lin3(x)
        x = self.act(x)
        x = self.lin4(x)
        return torch.sigmoid(x)
    #"""
    #def forward(self, t):
    #    print(t)
    #    return torch.exp(self.param[t])

class Curves(nn.Module): # put all 3 in 1
    def __init__(self, T = 1078):
        super().__init__()
        self.lin1 = nn.Linear(1, 48)
        self.lin2 = nn.Linear(48, 48)
        self.lin3 = nn.Linear(48, 48)
        self.lin4 = nn.Linear(48, 48)
        self.lin5 = nn.Linear(48, 3)

        self.act = torch.relu

        self.gamma = Greek()
        self.beta = Greek()
        self.mu = Greek()

    def forward(self, x): # takes time and produces S, I, T estimates
        x = self.lin1(x)
        x = self.act(x)
        x = self.lin2(x)
        x = self.act(x)
        x = self.lin3(x)
        x = self.act(x)
        x = self.lin4(x)
        x = self.act(x)
        x = self.lin5(x)
        return Ntilde * x # so the model only has to predict something smaller

    def get_derivative(self, t):
        t.requires_grad = True
        x = self.forward(t)

        grad_s = torch.autograd.grad(x[:,0].sum(), t, create_graph=True)[0]
        grad_i = torch.autograd.grad(x[:,1].sum(), t, create_graph=True)[0]
        grad_d = torch.autograd.grad(x[:,2].sum(), t, create_graph=True)[0]
        
        return grad_s, grad_i, grad_d

In [ ]:
NN = Curves()
optimizer = RMSprop(NN.parameters(), lr = 1e-3)

EPOCHS = 500
dataset = DataLoader(TensorDataset(T, Y), batch_size = 20, shuffle = True)
losses = []

for i in range(EPOCHS):

    total_loss = 0.

    for t, y in dataset:
        optimizer.zero_grad()
        
        y_hat = NN(t)

        ## assemble losses
        ### distance from the model output to the actual curves
        curve_loss = torch.sum((y - y_hat)**2)

        ### pde losses
        dsdt, didt, dddt = NN.get_derivative(t)

        dsdt_hat = NN.gamma(t) * torch.exp(y_hat[:,2] - y_hat[:,0]).reshape(-1, 1) - NN.beta(t) * torch.exp(y_hat[:,1] - Ntilde).reshape(-1, 1)
        dddt_hat = NN.mu(t) * torch.exp(y_hat[:,1] - y_hat[:,2]).reshape(-1, 1)
        didt_hat = NN.beta(t) * torch.exp(y_hat[:,0] - Ntilde).reshape(-1, 1) - \
                    NN.mu(t) * torch.exp(y_hat[:,2] - y_hat[:,1]).reshape(-1, 1) - NN.gamma(t)

        dsdt_loss = torch.sum((dsdt - dsdt_hat)**2)
        dddt_loss = torch.sum((dddt - dddt_hat)**2)
        didt_loss = torch.sum((didt - didt_hat)**2)

        pde_loss = dddt_loss + dsdt_loss + didt_loss

        ### boundary losses (e.g. the sum of pdes should be zero and of total values should be Ntilde
        Ntilde_hat = torch.log(torch.exp(y_hat[:,0]) + torch.exp(y_hat[:,1]) + torch.exp(y_hat[:,2]))
        N_loss = torch.sum((Ntilde - Ntilde_hat.reshape(-1, 1))**2)

        pde_zero_loss = torch.sum(dsdt + didt + dddt)

        boundary_loss = pde_zero_loss + N_loss

        ### smoothness losses (for later)

        ## total loss assemble
        l = 0.001 * boundary_loss + 0.001 * pde_loss + curve_loss
        
        l.backward()
        optimizer.step()

        total_loss += l.item() #.detach()

    losses.append(total_loss)

    if i % 9 == 0:
        print('=== Epoch', i, '| Loss:', total_loss, '===')
    


=== Epoch 0 | Loss: 23726.30544717584 ===
=== Epoch 9 | Loss: 1261.7454396258725 ===
=== Epoch 18 | Loss: 1465.9548098338341 ===
=== Epoch 27 | Loss: 1499.5829757781562 ===
=== Epoch 36 | Loss: 1329.2925023779687 ===
=== Epoch 45 | Loss: 1165.7422980813778 ===
=== Epoch 54 | Loss: 976.1905300184015 ===
=== Epoch 63 | Loss: 995.3568725637073 ===
=== Epoch 72 | Loss: 1012.0951560996192 ===
=== Epoch 81 | Loss: 971.5905702448149 ===
=== Epoch 90 | Loss: 903.0005329388374 ===
=== Epoch 99 | Loss: 934.3825372021099 ===
=== Epoch 108 | Loss: 859.866392525537 ===
=== Epoch 117 | Loss: 801.9227845333226 ===
=== Epoch 126 | Loss: 852.2088364555306 ===
=== Epoch 135 | Loss: 831.3289139200283 ===
=== Epoch 144 | Loss: 818.5814267752387 ===
=== Epoch 153 | Loss: 793.4034376031128 ===
=== Epoch 162 | Loss: 752.6178910422724 ===
=== Epoch 171 | Loss: 736.3343194247049 ===
=== Epoch 180 | Loss: 776.1770572752189 ===
=== Epoch 189 | Loss: 706.7675797190374 ===


In [ ]:
plt.plot(losses[10:])

In [ ]:
with torch.no_grad():
    plt.plot(NN(T)[:,2])
    plt.plot(Dtilde)

In [ ]:
with torch.no_grad():
    plt.plot(NN(T)[:,1])
    plt.plot(Itilde)

In [ ]:
with torch.no_grad():
    plt.plot(NN(T)[:,0])
    plt.plot(Stilde)

In [ ]:
t = torch.linspace(0, 1, 100).reshape(-1, 1)

with torch.no_grad():
    plt.plot(NN.beta(t)[:,0])